In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, r2_score, confusion_matrix, roc_curve, auc
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Load German Credit data
data = pd.read_csv(r'C:/UP/2024/Research project/data/german credit.csv')

# Clean column names
data.columns = [col.replace(' ', '.').replace('&','.').replace('/','.') for col in data.columns]

# Preprocess data
def preprocess_data(data):
    X = data.drop('Creditability', axis=1).values  
    y = data['Creditability'].values 
    # Standardize 
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X) 

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

class LBSA:
    def __init__(self, n_spiders=50, max_iterations=35, ra=1, pc=0.5, pm=0.5, rho=0.8, phi=0.2):
        self.n_spiders = n_spiders 
        self.max_iterations = max_iterations
        self.ra = ra 
        self.pc = pc 
        self.pm = pm 
        self.rho = rho 
        self.phi = phi 
        self.n_obl_spiders = int(0.1 * n_spiders)  
        self.n_ilsa_spiders = 10
        np.random.seed(2)

    def initialize_population(self, n_features):
        population = np.random.randint(2, size=(self.n_spiders, n_features)) 
        dimension_masks = np.zeros((self.n_spiders, n_features)) 
        
        # Apply OBL method
        opposite_population = 1 - population 
        all_solutions = np.vstack((population, opposite_population))
        fitness_values = np.array([self.fitness_function(X_train, y_train, ind) for ind in all_solutions])
        
        # Select the best solutions
        best_indices = np.argsort(fitness_values)[:self.n_spiders] 
        population = all_solutions[best_indices] 
        
        return population, dimension_masks

    def fitness_function(self, X, y, individual):
        selected_features = X[:, individual.astype(bool)]
        if selected_features.shape[1] == 0:
            return float('inf') 
        clf = GaussianNB() 
        clf.fit(selected_features, y)
        y_pred = clf.predict(selected_features) 
        error_rate = 1 - accuracy_score(y, y_pred) 
        n_selected_features = np.sum(individual) 
        return self.rho * error_rate + self.phi * (n_selected_features / len(individual))

    def calculate_vibration(self, fitness, source, target):
        intensity = np.log(1 / (fitness + 1e-10)) + 1 
        distance = np.sum(np.abs(source - target))
        sigma = np.std(source)
        return intensity * np.exp(-distance / (sigma * self.ra))

    def random_walk(self, spider, target_position, last_position):
        r = np.random.rand() 
        new_position = spider + (spider - last_position) * r + (target_position - spider) * np.random.rand(len(spider))
        return np.round(new_position).clip(0, 1).astype(int) 

    def fit(self, X, y):
        n_features = X.shape[1] 
        population, dimension_masks = self.initialize_population(n_features)
        last_positions = np.zeros_like(population) 
        target_vibrations = np.zeros(self.n_spiders) 
        
        for _ in range(self.max_iterations): 
            fitness_values = np.array([self.fitness_function(X, y, ind) for ind in population])
            best_index = np.argmin(fitness_values)
            
            # Generate and propagate vibrations
            for i in range(self.n_spiders): 
                vibration = self.calculate_vibration(fitness_values[i], population[i], population[i])
                for j in range(self.n_spiders):
                    if i != j:
                        received_vibration = self.calculate_vibration(fitness_values[i], population[i], population[j])
                        if received_vibration > target_vibrations[j]:
                            target_vibrations[j] = received_vibration
            
            # Apply ILSA
            best_indices = np.argsort(fitness_values)[:self.n_ilsa_spiders]
            
            for i in range(self.n_spiders):
                if i not in best_indices:
                    if np.random.rand() > self.pc:
                        dimension_masks[i] = np.random.rand(n_features) < self.pm
                    
                    target_position = np.where(dimension_masks[i], np.random.randint(2, size=n_features), population[best_index])
                    population[i] = self.random_walk(population[i], target_position, last_positions[i])
                    last_positions[i] = population[i].copy()
        
        best_solution = population[np.argmin([self.fitness_function(X, y, ind) for ind in population])]
        return best_solution
    
 # Main execution
X_train, X_test, y_train, y_test = preprocess_data(data)

lbsa = LBSA(n_spiders=50, max_iterations=35, ra=1, pc=0.5, pm=0.5, rho=0.8, phi=0.2)
best_features = lbsa.fit(X_train, y_train)

# Evaluate the model
selected_features_train = X_train[:, best_features.astype(bool)]
selected_features_test = X_test[:, best_features.astype(bool)]

clf = GaussianNB()
clf.fit(selected_features_train, y_train) 
y_pred = clf.predict(selected_features_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

feature_names = data.drop('Creditability', axis=1).columns
selected_feature_names = feature_names[best_features.astype(bool)]
print("Selected features:")
for feature in selected_feature_names:
    print(feature)

# Test accuracy with specified 9 features
specified_features = ['Account.Balance', 'Payment.Status.of.Previous.Credit', 'Value.Savings.Stocks', 'Sex...Marital.Status', 'Credit.Amount', 'Instalment.per.cent', 'Duration.of.Credit.(month)']

specified_features_mask = np.isin(feature_names, specified_features)
specified_features_train = X_train[:, specified_features_mask]
specified_features_test = X_test[:, specified_features_mask]

clf_specified = GaussianNB()
clf_specified.fit(specified_features_train, y_train)
y_pred_specified = clf_specified.predict(specified_features_test)

accuracy_specified = accuracy_score(y_test, y_pred_specified)
print(f"\nAccuracy with specified 7 features: {accuracy_specified:.4f}")

# Test accuracy with all features
clf_all = GaussianNB()
clf_all.fit(X_train, y_train)
y_pred_all = clf_all.predict(X_test)

accuracy_all = accuracy_score(y_test, y_pred_all)
print(f"\nAccuracy with all features: {accuracy_all:.4f}")

# Compare the results
print("\nComparison of accuracies:")
print(f"LBSA selected features: {accuracy:.4f}")
print(f"Specified 9 features: {accuracy_specified:.4f}")
print(f"All features: {accuracy_all:.4f}")


# Calculate metrics
y_prob = clf.predict_proba(selected_features_test)[:, 1]
y_pred = (y_prob > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

plt.figure(figsize=(15, 5))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix (LBSA)')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.savefig('CM_LBSA.pdf')
plt.show()

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='grey', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve - LBSA')
plt.legend(loc="lower right")
plt.savefig('ROC_LBSA.pdf')
plt.show()


# Cross-validation
cv_scores_lbsa = cross_val_score(clf, selected_features_train, y_train, cv=5)
print("\nLBSA Cross-validation scores:")
print(f"Mean: {cv_scores_lbsa.mean():.4f} (+/- {cv_scores_lbsa.std() * 2:.4f})")


# Learning Curve
train_sizes, train_scores, test_scores = learning_curve(
    clf, selected_features_train, y_train, cv=5, n_jobs=-1, 
    train_sizes=np.linspace(0.1, 1.0, 10))

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, color='red', marker='o', markersize=5, label='Training accuracy')
plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='red')
plt.plot(train_sizes, test_mean, color='green', linestyle='--', marker='s', markersize=5, label='Validation accuracy')
plt.fill_between(train_sizes, test_mean + test_std, test_mean - test_std, alpha=0.15, color='green')
plt.xlabel('Number of training samples')
plt.ylabel('Accuracy')
plt.title('Learning Curve - LBSA')
plt.legend(loc='lower right')
plt.savefig('LC_LBSA.pdf')
plt.show()  

# Calculate training accuracy
y_train_pred = clf.predict(selected_features_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy: {train_accuracy:.4f}")

# testing accuracy 
print(f"Testing Accuracy: {accuracy:.4f}")